In [172]:
import pandas as pd
from datetime import datetime
from pandas import DataFrame
from tabulate import tabulate
import time

In [168]:
template_data_set = 'data: [{0}].sort(function (a, b) {{return a.value - b.value;}}),'
template_data = '{{value: {0}, name: \'{1}\', tooltip: {{formatter: \'{{b}}:<br/>{2}total: {{c}}kg\'}}}},'
template_item = '{0}kg × {1}<br/>'
pd.set_option('max_colwidth', 2000)
pd.set_option('display.unicode.east_asian_width', True)

In [182]:
data = pd.read_csv('../datas/trains.csv', encoding='gbk')

def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

def filter_pd_data(_type, raw):
    ret = {}
    for index, row in raw.iterrows():
        key = row[_type]
        if key not in ret:
            ret[key] = []
        ret[key].append(row)
    return ret
data_by_date = filter_pd_data('DATE', data)

data_formatter = {}
for key, value in data_by_date.items():
    data_set_str = ''
    df = pd.DataFrame(value)
    data_by_name = filter_pd_data('NAME', df)
    for name, row in data_by_name.items():
        total = 0
        item_str = ''
        for item in row:
            weight = item['WEIGHT']
            times = item['TIMES']
            total += weight * times
            item_str += template_item.format(weight, times)
        data_set_str += template_data.format(total, name, item_str)
    data_formatter[key] = template_data_set.format(data_set_str)
    
formatter_data = pd.DataFrame(data_formatter.items(), columns=['Date', 'Formatter'])
formatter_data.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
# left_align(formatter_data)
from IPython.core.display import HTML
display(HTML(formatter_data.to_html(index=False, justify='left')))

Date,Formatter
2021/11/25,"data: [{value: 7200, name: '腿举', tooltip: {formatter: '{b}:<br/>120kg × 10<br/>140kg × 10<br/>140kg × 10<br/>160kg × 10<br/>160kg × 10<br/>total: {c}kg'}},{value: 2120, name: '直腿硬拉', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>80kg × 8<br/>100kg × 3<br/>100kg × 4<br/>100kg × 3<br/>total: {c}kg'}},{value: 2980, name: '自由深蹲', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>80kg × 8<br/>100kg × 3<br/>100kg × 3<br/>100kg × 3<br/>60kg × 8<br/>60kg × 8<br/>total: {c}kg'}},{value: 1632, name: '坐姿腿屈伸', tooltip: {formatter: '{b}:<br/>34kg × 12<br/>34kg × 12<br/>34kg × 12<br/>34kg × 12<br/>total: {c}kg'}},{value: 3168, name: '罗马挺', tooltip: {formatter: '{b}:<br/>66kg × 12<br/>66kg × 12<br/>66kg × 12<br/>66kg × 12<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}),"
2021/11/29,"data: [{value: 2300, name: '仰卧杠铃卧推', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>60kg × 8<br/>60kg × 8<br/>60kg × 6<br/>60kg × 6<br/>70kg × 2<br/>total: {c}kg'}},{value: 1980, name: '上斜器械推举', tooltip: {formatter: '{b}:<br/>30kg × 12<br/>30kg × 12<br/>30kg × 12<br/>40kg × 10<br/>50kg × 10<br/>total: {c}kg'}},{value: 1382, name: '窄距哑铃推举', tooltip: {formatter: '{b}:<br/>30kg × 12<br/>31kg × 12<br/>32kg × 10<br/>33kg × 10<br/>total: {c}kg'}},{value: 2600, name: '双杠臂屈伸', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>65kg × 10<br/>65kg × 10<br/>65kg × 10<br/>total: {c}kg'}},{value: 1100, name: '站姿高位拉锁夹胸', tooltip: {formatter: '{b}:<br/>30kg × 10<br/>30kg × 10<br/>30kg × 10<br/>20kg × 10<br/>total: {c}kg'}},{value: 2925, name: '悬垂卷腹', tooltip: {formatter: '{b}:<br/>65kg × 15<br/>65kg × 15<br/>65kg × 15<br/>total: {c}kg'}},{value: 2475, name: '跪姿拉索卷腹', tooltip: {formatter: '{b}:<br/>55kg × 15<br/>55kg × 15<br/>55kg × 15<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}),"
2021/11/30,"data: [{value: 2745, name: '宽握引体向上', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>65kg × 8<br/>65kg × 8<br/>65kg × 7<br/>75kg × 4<br/>75kg × 4<br/>total: {c}kg'}},{value: 2400, name: '俯身杠铃划船', tooltip: {formatter: '{b}:<br/>60kg × 10<br/>60kg × 10<br/>60kg × 10<br/>60kg × 10<br/>total: {c}kg'}},{value: 4250, name: '坐姿拉索划船', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>75kg × 12<br/>75kg × 12<br/>75kg × 12<br/>75kg × 12<br/>total: {c}kg'}},{value: 2800, name: '高位宽握下拉', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>75kg × 10<br/>75kg × 10<br/>65kg × 10<br/>total: {c}kg'}},{value: 2000, name: '器械坐姿划船', tooltip: {formatter: '{b}:<br/>40kg × 10<br/>40kg × 10<br/>40kg × 10<br/>40kg × 10<br/>20kg × 20<br/>total: {c}kg'}},{value: 750, name: '直臂下压', tooltip: {formatter: '{b}:<br/>15kg × 10<br/>15kg × 10<br/>20kg × 10<br/>25kg × 10<br/>total: {c}kg'}},{value: 2250, name: '坐姿卷腹', tooltip: {formatter: '{b}:<br/>30kg × 25<br/>30kg × 25<br/>30kg × 25<br/>total: {c}kg'}},{value: 1000, name: '俄式转体', tooltip: {formatter: '{b}:<br/>10kg × 50<br/>10kg × 50<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}),"


In [ ]:
formatter_data[formatter_data['Date'] == time.strftime('%Y/%m/%d', time.localtime(time.time()))]['Formatter']